In [2]:
import pandas as pd
import os

# In a notebook, __file__ doesn't exist, so use relative path
BASE_DIR = os.path.abspath("..")  # parent folder of /notebooks
csv_path = os.path.join(BASE_DIR, "data", "processed", "cpi_food_clean.csv")

df = pd.read_csv(csv_path)
df.head()

# Convert date to datetime (if not already)
df["date"] = pd.to_datetime(df["date"])

# Sort correctly
df = df.sort_values(["category", "date"])

# Calculate MoM % change for each category
df["mom_pct_change"] = df.groupby("category")["value"].pct_change() * 100

df.head()


,date,series_id,category,value,mom_pct_change
0,2015-01-01,CUSR0000SEFC,Cereals & bakery,325.728,NaN
1,2015-02-01,CUSR0000SEFC,Cereals & bakery,326.521,0.243455
2,2015-03-01,CUSR0000SEFC,Cereals & bakery,324.972,-0.474395
3,2015-04-01,CUSR0000SEFC,Cereals & bakery,325.895,0.284024
4,2015-05-01,CUSR0000SEFC,Cereals & bakery,324.713,-0.362694
